# Replicate Rivkin for Layered Landscapes

In [ ]:
##Import packages and files
from itertools import compress, product
import numpy as np
import matplotlib.pyplot as plt
import array
import random
import pandas as pd
import itertools
import operator
import importlib
#import functions defining the layered landscape
from LayeredLandscapeFunctions import *


In [ ]:
Position_rand(10)

In [ ]:
def combinations(items):
    return ( list(compress(items,mask)) for mask in product(*[[0,1]]*len(items)) )
    # alternative:      

In [ ]:
#Get_Fitness_Layered_Landscape(Landscape1, Landscape2, Landscape_Weights, Position)

In [ ]:
## Create an algorithm of search so that we select M from N attirbutes to change,
# then stop when we find an improvment


## Inputs: Landscape, iPosition, M
## Landscape1 - Input of landscape1 
## Landscape2 - Input of landscape2
## iPosition - inital postition across all variables
## M - M, number of genes to search over
### Outputs:  NewPosition
## NewPosition - The new optimal position found

###NOTE Be vary careful as with high N and M this becomes exponentially longer

def Incremental_Improvement(iPosition, Landscape1, Landscape2, Landscape_Weights, M):
    #initialize variables
    ChangedGenes = np.array(iPosition[:])
    N=len(iPosition) # OR N = len(iPosition)
    ##Get current fitness and oldFitness
    OldFit = Get_Fitness_Layered_Landscape(Landscape1, Landscape2, Landscape_Weights, iPosition) #get_fitness_Product_World(Landscape,iPosition)

    CurrentFit = Get_Fitness_Layered_Landscape(Landscape1, Landscape2, Landscape_Weights, iPosition) #get_fitness_Product_World(Landscape,iPosition)
    #Create list of all permutations of 0,1 of length M
    for j in range(M+1):
            #Get a list of zeros
            if j==0:
                all_combos=np.zeros((1,M),dtype="int8")
            if j>=1:
                #this works, not fully understood by me, but thank god for stackoverflow.
                which = np.array(list(itertools.combinations(range(M), j)))
                #set a grid of zeros
                grid = np.zeros((len(which), M), dtype="int8")
                #change
                grid[np.arange(len(which))[None].T, which] = 1
                #Magic
                all_combos = np.concatenate((all_combos, grid))
                #Now we have a list of all possible permutations of fitness of length M stored in all_combos

    #What remains is to loop over (up to all) combinations of M indices from our N genes 
    #(we can do combinations here since we did permutations before)

    All_indici_combos = list(itertools.combinations(range(N), M))
    ##now loop over the two, getting the fitness of a new  combo
    for indici in All_indici_combos:
        for gene_combo in all_combos:
            ChangedGenes = np.array(iPosition[:])
            ChangedGenes[list(indici)] = gene_combo 
            #See if new fitness greater than old, if so then we are done
            ####THIS WAS UNCLEAR IN RIVKIN, PERHAPS I OUGHT TO RETURN THE MAX.
            NewFitness=Get_Fitness_Layered_Landscape(Landscape1, Landscape2, Landscape_Weights, ChangedGenes)
            if NewFitness > CurrentFit:
                CurrentFit = NewFitness
                CurrentPosition = ChangedGenes
    #if no change than return initial position
    if OldFit > CurrentFit:
        return(iPosition)
    else:
        return(CurrentPosition)


    ##GOOD IDEA: Should also return another variable if no change that says it is at a local optima.
    #That way we know not to loop over this long algorithm again.

    ###Note: This code is actually not the quickest possible, but it does work. We could make it more efficient
    # so that duplicates do not occur (I won't though, not worth my time)


In [117]:
## Create an algorithm of imitation to copy J of N attributes of best performer with success probability Theta
#then stop when we find an improvment


## Inputs: iPosition, Landscape, iPosition, J,Theta
## iPosition - inital postition across all variables
## Landscape1 - Input of landscape1 values
## Landscape2 - Input of landscape2 values
## Landscape_Weights - Weights for Layering Landscape 1 and two
## bestPosition - Position of Best agent
## J -  number of genes to copy
## Theta -  Probability of Successful copy of each individaul gene/decision

### Outputs:  NewPosition
## NewPosition - The new optimal position found


def follow_the_leader(iPosition, Landscape1, Landscape2, Landscape_Weights, bestPosition, J, Theta):
    #check to see if fitness worse
    if (Get_Fitness_Layered_Landscape(Landscape1, Landscape2, Landscape_Weights, iPosition) <
        Get_Fitness_Layered_Landscape(Landscape1, Landscape2, Landscape_Weights, bestPosition)):
        #reset the best genes
        bestGenes = bestPosition
        #Randomly select J of the bestPosition Genes
        indices_of_best = np.random.choice(bestPosition.shape[0], J, replace=False)  
        randomGenes = bestPosition[indices_of_best]
        #now for each, replicate poorly with probability theta
        #draw array from U[0,1] of size J
        Probs = np.random.uniform(size=J)
        #Get array determing replicated genes
        Success = Probs<Theta
        #Get replicated genes
        Replicated = abs(randomGenes+(Success-1))
        #Set initial genes to the replicated ones
        iPosition[indices_of_best]=Replicated
        return(iPosition)
   ##if no better return the initial position
    else:
        return(iPosition)
    
    
#like follow the leader, but you must go to it.
def go_to_the_leader(iPosition, Landscape1, Landscape2, Landscape_Weights, bestPosition, J, Theta):
        #reset the best genes
        bestGenes = bestPosition
        #Randomly select J of the bestPosition Genes
        indices_of_best = np.random.choice(bestPosition.shape[0], J, replace=False)  
        randomGenes = bestPosition[indices_of_best]
        #now for each, replicate poorly with probability theta
        #draw array from U[0,1] of size J
        Probs = np.random.uniform(size=J)
        #Get array determing replicated genes
        Success = Probs<Theta
        #Get replicated genes
        Replicated = abs(randomGenes+(Success-1))
        #Set initial genes to the replicated ones
        iPosition[indices_of_best]=Replicated
        return(iPosition)




In [118]:
## Create an algorithm of hybrid search that replicates rivkin in Table 7


## Inputs: iPosition, Landscape, iPosition, J,Theta
## iPosition - inital postition across all variables
## Landscape1 - Input of landscape1 values
## Landscape2 - Input of landscape2 values
## Landscape_Weights - Vector of weights for layered landscapes
## bestPosition - Position of Best agent
## J -  number of genes to copy
## Theta -  Probability of Successful copy of each individaul gene/decision
## M - M, number of genes to search over for incremental Improvement


### Outputs:  NewPosition
## NewPosition - The new optimal position found


def hybrid_climb_continuos(iPosition,Landscape1,Landscape2,Landscape_Weights,benchmark_position,J,Theta,M):
    ##First of all, do an upward hill climb via incremental improvement, but check for potential improvements
    #Set initial genes
    InitialGenes = np.array(iPosition[:])

    #set the current position based upon a first step of going to the leader
    CurrentPosition = go_to_the_leader(InitialGenes, Landscape1, Landscape2, Landscape_Weights,
                                        benchmark_position, J, Theta)
    
    
    #set fitnesses initially
    currentFitness = Get_Fitness_Layered_Landscape(Landscape1, Landscape2, Landscape_Weights, iPosition) 
    OldFitness = Get_Fitness_Layered_Landscape(Landscape1, Landscape2, Landscape_Weights, CurrentPosition)
    NewGenes = Incremental_Improvement(CurrentPosition, Landscape1, Landscape2, Landscape_Weights, M)
    NewFitness = Get_Fitness_Layered_Landscape(Landscape1, Landscape2, Landscape_Weights, NewGenes)
    
    #keep searching until you reach a maxima
    while(NewFitness!=OldFitness):
        #reset the old fitness
        ###Okay so this is inefficient because we are calling the function again, but I have had issues where
        #OldFitness and NewFitness are stored in memory 
        #OldFitness = Get_Fitness_Layered_Landscape(Landscape1, Landscape2, Landscape_Weights, NewGenes) #get_fitness_Product_World(Landscape,NewGenes)
        OldFitness = NewFitness
        NewGenes = Incremental_Improvement(NewGenes, Landscape1, Landscape2, Landscape_Weights, M)#Incremental_Improvement(NewGenes, Landscape,M)
        
        NewFitness =  Get_Fitness_Layered_Landscape(Landscape1, Landscape2, Landscape_Weights, NewGenes)#get_fitness_Product_World(Landscape,NewGenes)
    return(NewGenes)
   

In [120]:
## Define a function to replciate table 7 results
#Inputs:
#N - N
#K2- the Second Landscape K (first is 0)
#Sims - Number of Simulated Landscapes to generate statistics
def RunTable7Sims(K2,Sims,N):
    ArrayCountNewBenchmarks = []
    ArrayCountReachedBenchmarks = []
    ArrayMeanTimeSinceLastMeetExceed = []
    for i in range(Sims):
        ##First define landscapes and other variables
        K1=0
        J=int(N/2)
        Theta = .9
        M=1
        (Landscape1, Landscape2) = Generate_Layered_Landscapes(N,K1,K2)
        Landscape_Weights = [.6666,.3333]
        # I believe I am not supposed to have a random search over 100 agents first.... like in table 6
        #initialize null benchmarks
        benchmark_fitness = 0
        benchmark_position = 0

        ##Set up counters
        CountNewBenchmarks = 0
        CountReachedBenchmarks = 0
        DurationsTillMeetExceed = []
        TimeSinceLastMeetExceed = 0
        ##Now loop over 1000 firms
        for firm in range(1000):
            init_firm_position = Position_rand(N)
            ##do not benchmark for first firm, instead just go to local max
            if firm==0:
                OldFitness = 0
                currentFit =  Get_Fitness_Layered_Landscape(Landscape1, Landscape2,
                                                            Landscape_Weights, init_firm_position)
                new_firm_position=init_firm_position
                ##loop over fitness improvements until maxima
                while(currentFit!=OldFitness):
                    OldFitness = currentFit
                    new_firm_position = Incremental_Improvement(new_firm_position, Landscape1, Landscape2,
                                                       Landscape_Weights, M)
                    currentFit =  Get_Fitness_Layered_Landscape(Landscape1, Landscape2,
                                                                Landscape_Weights, new_firm_position)
                    #print(OldFitness,currentFit)
                #the first firm sets a benchmark

            else:
                new_firm_position = hybrid_climb_continuos(init_firm_position,Landscape1,Landscape2,
                                                       Landscape_Weights,benchmark_position,J,Theta,M)
            currentFit = Get_Fitness_Layered_Landscape(Landscape1, Landscape2, Landscape_Weights, new_firm_position)
            ##update time till exceed counter
            TimeSinceLastMeetExceed+=1
            #print(init_firm_position)
            ##Check if we reach the old benchmark
            #print(TimeSinceLastMeetExceed)
            if  currentFit == benchmark_fitness:
               # print('reached')
                #print('Reach Benchmark')
                CountReachedBenchmarks += 1
                ##update time counter
                DurationsTillMeetExceed.append(TimeSinceLastMeetExceed)
                TimeSinceLastMeetExceed = 0
            #check if new fitness meets benchmark, then reset benchmark
            if  currentFit > benchmark_fitness:
                #print('surpassed')
                benchmark_fitness = currentFit
                benchmark_position = new_firm_position
                CountNewBenchmarks += 1
                ##update time counter
                DurationsTillMeetExceed.append(TimeSinceLastMeetExceed)
                TimeSinceLastMeetExceed = 0
            #if firm % 300 == 1:
                #print(firm) ##this helps us keep track of what is going on
        #print('Finished')
        MeanTimeSinceLastMeetExceed=sum(DurationsTillMeetExceed)/len(DurationsTillMeetExceed)
        ArrayMeanTimeSinceLastMeetExceed.append(MeanTimeSinceLastMeetExceed)
        ArrayCountReachedBenchmarks.append(CountReachedBenchmarks)
        ArrayCountNewBenchmarks.append(CountNewBenchmarks)

        #print('How many times is an existing benchmark matched (but not exceeded) by a subsequent firm:',
        #      CountReachedBenchmarks)
        #print('How many times is a new benchmark established :',CountNewBenchmarks)
        #print('Average number of firms before new benchmark is matched or exceeded',MeanTimeSinceLastMeetExceed)
        
    MeanTimeToBench = sum(ArrayMeanTimeSinceLastMeetExceed)/len(ArrayMeanTimeSinceLastMeetExceed)
    MeanCountReached = sum(ArrayCountReachedBenchmarks)/len(ArrayCountReachedBenchmarks)
    MeanCountNew = sum(ArrayCountNewBenchmarks)/len(ArrayCountNewBenchmarks)
   
    
    print('N =',N, 'K =',K2)
    print('How many times is a new benchmark established on average :',MeanCountNew)
    print('How many times is an existing benchmark matched (but not exceeded) by a subsequent firm on average:',
          MeanCountReached)
    print('Average number of firms before new benchmark is matched or exceeded',MeanTimeToBench)
###Note the average does make sense since it is averaged over sims and the denominator is changing...
###don't worry - the math checks out


In [107]:
 RunTable7Sims(K2=1,Sims=20,N=12)

N = 12 K = 1
How many times is a new benchmark established on average : 1.3
How many times is an existing benchmark matched (but not exceeded) by a subsequent firm on average: 798.7
Average number of firms before new benchmark is matched or exceeded 1.3641302319691737


In [108]:
 RunTable7Sims(K2=3,Sims=20,N=12)

N = 12 K = 3
How many times is a new benchmark established on average : 2.1
How many times is an existing benchmark matched (but not exceeded) by a subsequent firm on average: 526.1
Average number of firms before new benchmark is matched or exceeded 2.2375451006418894


In [109]:
 RunTable7Sims(K2=7,Sims=20,N=12)


N = 12 K = 7
How many times is a new benchmark established on average : 3.15
How many times is an existing benchmark matched (but not exceeded) by a subsequent firm on average: 268.6
Average number of firms before new benchmark is matched or exceeded 4.1594955794669755


In [121]:
 RunTable7Sims(K2=11,Sims=20,N=12)


N = 12 K = 11
How many times is a new benchmark established on average : 3.75
How many times is an existing benchmark matched (but not exceeded) by a subsequent firm on average: 165.95
Average number of firms before new benchmark is matched or exceeded 8.172537442150439


### end of N=12

## Below is N=8

In [122]:
 RunTable7Sims(K2=1,Sims=20,N=8)


N = 8 K = 1
How many times is a new benchmark established on average : 1.15
How many times is an existing benchmark matched (but not exceeded) by a subsequent firm on average: 857.8
Average number of firms before new benchmark is matched or exceeded 1.2597646516589287


In [123]:
 RunTable7Sims(K2=3,Sims=20,N=8)


N = 8 K = 3
How many times is a new benchmark established on average : 1.6
How many times is an existing benchmark matched (but not exceeded) by a subsequent firm on average: 648.7
Average number of firms before new benchmark is matched or exceeded 1.6168040899529603


In [124]:
 RunTable7Sims(K2=7,Sims=20,N=8)


N = 8 K = 7
How many times is a new benchmark established on average : 2.8
How many times is an existing benchmark matched (but not exceeded) by a subsequent firm on average: 311.8
Average number of firms before new benchmark is matched or exceeded 4.418954134523311


In [125]:
 RunTable7Sims(K2=1,Sims=20,N=4)


N = 4 K = 1
How many times is a new benchmark established on average : 1.15
How many times is an existing benchmark matched (but not exceeded) by a subsequent firm on average: 932.1
Average number of firms before new benchmark is matched or exceeded 1.1012842957010842


In [126]:
 RunTable7Sims(K2=3,Sims=20,N=4)


N = 4 K = 3
How many times is a new benchmark established on average : 1.25
How many times is an existing benchmark matched (but not exceeded) by a subsequent firm on average: 813.5
Average number of firms before new benchmark is matched or exceeded 1.3588573330048446


In [127]:
RunTable7Sims(K2=1,Sims=20,N=24)


N = 24 K = 1
How many times is a new benchmark established on average : 1.25
How many times is an existing benchmark matched (but not exceeded) by a subsequent firm on average: 769.15
Average number of firms before new benchmark is matched or exceeded 1.3811999798550192


In [128]:
 RunTable7Sims(K2=3,Sims=20,N=24)


N = 24 K = 3
How many times is a new benchmark established on average : 3.25
How many times is an existing benchmark matched (but not exceeded) by a subsequent firm on average: 294.1
Average number of firms before new benchmark is matched or exceeded 5.473944203521848


In [129]:
 RunTable7Sims(K2=7,Sims=20,N=24)


N = 24 K = 7
How many times is a new benchmark established on average : 3.5
How many times is an existing benchmark matched (but not exceeded) by a subsequent firm on average: 124.9
Average number of firms before new benchmark is matched or exceeded 10.831834050451398


In [130]:
 RunTable7Sims(K2=11,Sims=20,N=24)


N = 24 K = 11
How many times is a new benchmark established on average : 5.4
How many times is an existing benchmark matched (but not exceeded) by a subsequent firm on average: 41.15
Average number of firms before new benchmark is matched or exceeded 27.161119948139145


In [131]:
arr24new =[1.25,3.25,3.5,5.4]
arr24meet =[1.25,3.25,3.5,5.4]
arr24timetillmeet =[10.83,27.16]

In [ ]:
#####Replciate rivking table 7
##First define landscapes and other variables
N=24
K1=0
K2=11
J=N

Theta = .9
M=1
(Landscape1, Landscape2) = Generate_Layered_Landscapes(N,K1,K2)
Landscape_Weights = [.6666,.3333]

##Create Data to replicate Rivkin Table 7 Panels A and B

##Setup to find maximal fitness over 100 exploratory searches, this can be a little time consuming
Agent_Positions = [Position_rand(N) for _ in range(100)]
#FitnessMat = pd.DataFrame([Get_Fitness_Layered_Landscape(Landscape1, Landscape2, Landscape_Weights, X) for X in Agent_Positions])
Fitnesses = [Get_Fitness_Layered_Landscape(Landscape1, Landscape2, Landscape_Weights, X) for X in Agent_Positions]
bestPositionIndex,value = max(enumerate(Fitnesses), key=operator.itemgetter(1))
#get benchmark
initial_benchmark_position = Agent_Positions[bestPositionIndex]
benchmark_position = initial_benchmark_position
benchmark_fitness = Get_Fitness_Layered_Landscape(Landscape1, Landscape2, Landscape_Weights, benchmark_position)

print(benchmark_fitness)

##Set up counters
CountNewBenchmarks = 0
CountReachedBenchmarks = 0
DurationsTillMeetExceed = []
TimeSinceLastMeetExceed = 0
##Now loop over 1000 firms
for firm in range(1000):

    init_firm_position = Position_rand(N)
    new_firm_position = hybrid_climb_continuos(init_firm_position,Landscape1,Landscape2,
                                           Landscape_Weights,benchmark_position,J,Theta,M)


    currentFit = Get_Fitness_Layered_Landscape(Landscape1, Landscape2, Landscape_Weights, new_firm_position)
    ##update time till exceed counter
    TimeSinceLastMeetExceed+=1
    #print(init_firm_position)
    ##Check if we reach the old benchmark
    if  currentFit == benchmark_fitness:
        #print('Reach Benchmark')
        CountReachedBenchmarks += 1
        ##update time counter
        DurationsTillMeetExceed.append(TimeSinceLastMeetExceed)
        TimeSinceLastMeetExceed = 0
    #check if new fitness meets benchmark, then reset benchmark
    if  currentFit > benchmark_fitness:
        benchmark_fitness = currentFit
        benchmark_position = new_firm_position
        CountNewBenchmarks += 1
        ##update time counter
        DurationsTillMeetExceed.append(TimeSinceLastMeetExceed)
        TimeSinceLastMeetExceed = 0
        #print('New Benchmark')
        #print(benchmark_fitness)

    if firm % 100 == 1:
        print(firm) ##this helps us keep track of what is going on


print('Finished')
print('How many times is an existing benchmark matched (but not exceeded) by a subsequent firm:',
      CountReachedBenchmarks)
print('How many times is a new benchmark established :',CountNewBenchmarks)
MeanTimeSinceLastMeetExceed=sum(DurationsTillMeetExceed)/len(DurationsTillMeetExceed)
print('Average number of firms before new benchmark is matched or exceeded',MeanTimeSinceLastMeetExceed)




